In [1]:
import numpy as np
import json 

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
data_path = 'data/HNC_001+A4Ac+MOS_25934.npz'
data_npz = np.load(data_path, allow_pickle=True)
data_dict = dict(data_npz)['arr_0'].item()
data_dict.keys()

dict_keys(['Submand-PTV', 'SpinalCord_05', 'SpinalCord', 'Shoulders', 'Posterior_Neck', 'Pituitary', 'PharynxConst', 'PharConst-PTV', 'Parotids-PTV', 'Parotids', 'ParotidIps-PTV', 'ParotidCon-PTV', 'PTV_Total', 'PTV70', 'OralCavity', 'OpticNerve_R', 'OpticNerve_L', 'OCavity-PTV', 'Mandible-PTV', 'Mandible', 'Lungs', 'Lips', 'Lens_R', 'Lens_L', 'Larynx-PTV', 'Larynx', 'LacrimalGlands', 'Eyes', 'Esophagus', 'Cochlea_R', 'Cochlea_L', 'Chiasm', 'BrainStem_03', 'BrainStem', 'Brain', 'BrachialPlexus', 'Body', 'SubmandL-PTV', 'SubmandR-PTV', 'Submandibular', 'Thyroid', 'Thyroid-PTV', 'Trachea', 'all_mask', 'img', 'dose', 'dose_scale', 'isVMAT', 'angle_plate', 'isocenter', 'angle_list', 'origin', 'spacing', 'direction', 'size', 'beam_plate', 'RingPTVHigh', 'RingPTVLow', 'PTVHighOPT', 'PTVLowOPT', 'PTVLow-PTVMid', 'PTVLow'])

# Metric 1: Mean Absolute Error

### Step 1: Normalize the dose by setting D97 to prescribed dose of PTV_High

We will do this normalization in validation and testing phase, please make sure your predicted dose normalized in the same way. 

In [3]:
scale_dose_Dict = json.load(open('PTV_DICT.json'))
ref_dose = data_dict['dose'] * data_dict['dose_scale']
ptv_highdose =  scale_dose_Dict['HNC_001']['PTV_High']['PDose']
norm_scale = ptv_highdose / (np.percentile(ref_dose[data_dict['PTVHighOPT'].astype('bool')], 3) + 1e-5)
ref_dose = ref_dose * norm_scale

### Step 2: Calculate the error

Considering 1) Higher dose region is usually more important in clinical practice and 2) only dose within the body is of the interest, we mask out the regions with the 5 Gy isodose line and body mask. 

In [11]:
prediction = np.random.random(ref_dose.shape) * 10 # replace random matrix with your prediction

isodose_5Gy_mask = ((ref_dose > 5) | (prediction > 5)) & (data_dict['Body'] > 0) # the mask include the body AND the region where the dose/prediction is higher than 5Gy

diff = ref_dose - prediction

error = np.mean(np.abs(diff)[isodose_5Gy_mask > 0])

print (f'the MAE error is {error:.3f}')

the MAE error is 17.917


In [12]:
np.abs(diff)[isodose_5Gy_mask > 0].shape, diff.shape

((529020,), (128, 137, 175))

# Metric 2: Plan quality of the derived deliverable plan

The plan quality will be evaluted by the organizer using the Eclipse planning system. The participants only need to submit their prediction or solution. The rationale and pipeline of how the deliverable plan being created and how the plan quality metric is calcuated are documented in Ref [1] of <a href="README.md" _target='blank'>README.md</a>. 